In [2]:
import autograd.numpy as np
import pyipopt
from autograd import grad, hessian, jacobian

In [3]:
import matplotlib.pyplot as plt
from counterfactual_functions import *
from helper import *

username='boraozaltun'

data_dict = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/final_data/data_20200212.pickle')

reg_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/reg_2_num.pickle')
comm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/comm_2_num.pickle')

n = data_dict['n']
g = data_dict['g']
k = data_dict['k']

data_dict['R_hat'] = np.ones((n, g))
X_0 = np.ones(n*g + n)*1.0

eval_g = lambda x: reduced_counterfactual(x, data_dict)
eval_f = lambda x: np.linalg.norm(reduced_counterfactual(x, data_dict))

In [4]:
nvar = X_0.shape[0]
x_L = np.zeros((nvar))
x_U = np.ones((nvar)) * 10000.0

ncon = nvar

g_L = np.zeros((nvar))*1.0
g_U = np.zeros((nvar))*1.0

eval_grad_f = grad(eval_f)


nnzj = nvar*nvar
def eval_jac_g(X, flag, user_data=None):
    """
    Evaluate the sparse Jacobian of constraint functions g.
    @param X: parameter values
    @param flag: this asks for the sparsity structure
    """
    print('eval_jac_g')
    print()
    #XXX
    if flag:
        rows = np.linspace(0, len(X) -1, len(X)).astype(int)
        cols = np.linspace(0, len(X) -1, len(X)).astype(int)
        return (rows, cols)
    else:
        
        return jacobian(eval_g)(X)

nnzh =  int((nvar*(nvar+1))/2)

In [6]:
nvar

3752

In [4]:
nlp = pyipopt.create(nvar, x_L, x_U, ncon, g_L, g_U, nnzj, nnzh, eval_f, eval_grad_f, eval_g, eval_jac_g)

In [ ]:
print("Going to call solve")
print("x0 = {}".format(X_0))
x, zl, zu, constraint_multipliers, obj, status = nlp.solve(X_0)



In [1]:
nnzj

NameError: name 'nnzj' is not defined